In [19]:
!pip install langchain_community langchain-openai duckduckgo-search ddgs

In [16]:
from google.colab import userdata
from langchain_community.llms import OpenAI

llm = OpenAI(
    base_url=userdata.get('baseurl'),
    api_key=userdata.get('apikey'),
    model='meta-llama/Llama-3.3-70B-Instruct',
  )


from langchain.chat_models import ChatOpenAI

llm_chat = ChatOpenAI(
    base_url=userdata.get('baseurl'),
    api_key=userdata.get('apikey'),
    model='meta-llama/Llama-3.3-70B-Instruct',
    temperature=0
  )

In [2]:
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun

searcher = DuckDuckGoSearchRun()
def search_tool(query: str) -> str:
    """Ищет в интернете с помощью DuckDuckGo и возвращает краткий результат."""
    return searcher.run(query)

tool_search = Tool.from_function(
    func=search_tool,
    name="search",
    description="Используется для веб-поиска: принимает строку-запрос и возвращает найденный текст."
)

In [15]:
from langchain.tools import Tool
from langchain.chains import LLMMathChain
from langchain_community.llms import OpenAI

# инициализируем LLMMathChain на базе OpenAI-LLM
llm_math = LLMMathChain.from_llm(llm)

def calc_tool(expression: str) -> str:
    """Вычисляет математическое выражение через LLMMathChain."""
    return llm_math.run(expression)

tool_calc = Tool.from_function(
    func=calc_tool,
    name="calculator",
    description="Выполняет точные математические вычисления."
)


In [5]:
import pandas as pd
from langchain.tools import Tool

def csv_parser(file_path: str) -> str:
    """
    Считывает CSV, возвращает первые 5 строк в виде строки.
    Полезно для обзора структуры и быстрых подсказок.
    """
    df = pd.read_csv(file_path)
    return df.head().to_csv(index=False)

tool_csv = Tool.from_function(
    func=csv_parser,
    name="csv_parser",
    description="Парсит CSV-файл и возвращает первые строки."
)


In [6]:
from langchain.memory import ConversationBufferMemory

memory_buffer = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


/tmp/ipython-input-6-3142955513.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_buffer = ConversationBufferMemory(


In [17]:
from google.colab import userdata

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

memory_summary = ConversationSummaryMemory(
    llm=llm_chat,
    memory_key="summary",
    llm_prefix="Assistant:"
)


In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import DuckDuckGoSearchRun

try:
    tools = [tool_search, tool_calc]
except ImportError:
    print("Warning: LLMMath could not be imported. The calculator tool will not be available.")
    tools = [tool_search]


agent_react = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory_buffer,
    verbose=True
)

# Пример запуска:
response = agent_react.run("Кто выиграл чемпионат мира по футболу 2022 и сколько лет прошло?")
print(response)

/tmp/ipython-input-18-697139883.py:12: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_react = initialize_agent(
/tmp/ipython-input-18-697139883.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_react.run("Кто выиграл чемпионат мира по футболу 2022 и сколько лет прошло?")




> Entering new AgentExecutor chain...
 Я не знаю, кто выиграл чемпионат мира по футболу 2022 года, но я знаю, что он проходил в Катаре. Чтобы узнать победителя, мне нужно поискать в интернете.
Action: search
Action Input: Победитель чемпионата мира по футболу 2022

/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Aug 22, 2024 · Вспоминаем всех победителей чемпионатов мира по футболу. Из-за разногласий с Международной федерацией футбола (далее – ФИФА) англичане не … 6 days ago · В первом раунде 12 худших по рейтингу команд ( занимают места с 35 по 46) сыграли на выбывание по принципу «дома, в гостях». 6 победителей игр: Монголия, Шри … Jun 28, 2025 · 18 декабря завершился чемпионат мира по футболу в Катаре. Его победителем стала сборная Аргентины, серебряным призером — команда Франции, а бронзовые … Nov 1, 2024 · Все матчи и результаты чемпионатов мира FIFA: групповой этап, плей-офф, 16-й раунд, четвертьфиналы, полуфиналы, плей-офф за третье место и финальный матч 2022 … Nov 12, 2024 · Английская сборная по футболу стала победителем Чемпионата мира 2022 года, который проходил в Катаре.
Thought:

/usr/local/lib/python3.11/dist-packages/langchain_community/llms/openai.py:469: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


...

/usr/local/lib/python3.11/dist-packages/langchain_community/llms/openai.py:469: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


 На основе моих исследований я считаю, что Аргентина выиграла чемпионат мира по футболу 2022 года, но мне нужно вычислить количество лет, прошедших с тех пор.
Action: calculator
Action Input: 2024-2022

/usr/local/lib/python3.11/dist-packages/langchain_community/llms/openai.py:469: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()



Observation: Answer: 2
Thought: Я теперь знаю окончательный ответ
Final Answer: Аргентина выиграла чемпионат мира по футболу 2022 года и прошло 2 года.

> Finished chain.
Аргентина выиграла чемпионат мира по футболу 2022 года и прошло 2 года.


/usr/local/lib/python3.11/dist-packages/langchain_community/llms/openai.py:469: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

# Шаг 1: генерируем план
plan_prompt = PromptTemplate(
    input_variables=["input"],
    template="Построй план из шагов для достижения цели: {input}\nПлан:\n"
)
plan_chain = LLMChain(llm=llm, prompt=plan_prompt, output_key="plan")

# Шаг 2: выполняем план
exec_prompt = PromptTemplate(
    input_variables=["plan"],
    template="У тебя есть план:\n{plan}\nПоследовательно выполни шаги и выдай результат."
)
exec_chain = LLMChain(llm=llm, prompt=exec_prompt, output_key="final_output")

agent_plan_execute = SequentialChain(
    chains=[plan_chain, exec_chain],
    input_variables=["input"],
    output_variables=["plan", "final_output"],
    verbose=True
)

# Пример запуска:
result = agent_plan_execute({"input": "Подготовить сводный отчёт по новостям AI за неделю"})
print(result["final_output"])


<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._boots



> Entering new SequentialChain chain...


/usr/local/lib/python3.11/dist-packages/langchain_community/llms/openai.py:469: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()



> Finished chain.
 

## Шаг 1: Сбор и фильтрация информации
Следуя новостным источникам, специализирующимся на ИИ, за последнюю неделю были собраны следующие новости:
- Запуск нового ИИ-ассистента для медицинских исследований.
- Развитие алгоритмов для более точного распознавания речи.
- Применение ИИ в системах автономного вождения.
- Публикация статьи о потенциальных рисках ИИ.

Отфильтровав по актуальности, важности и теме, остаются новости о запуске ИИ-ассистента, развитии алгоритмов распознавания речи и применении ИИ в автономном вождении.

## Шаг 2: Организация информации
Категоризация новостей по темам:
- **Разработки**: Запуск нового ИИ-ассистента для медицинских исследований, развитие алгоритмов для более точного распознавания речи.
- **Применения**: Применение ИИ в системах автономного вождения.

Упоряд


/usr/local/lib/python3.11/dist-packages/langchain_community/llms/openai.py:469: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


In [21]:
from langchain.chains import LLMChain

# Базовый генератор ответа
gen_prompt = PromptTemplate(
    input_variables=["input"],
    template="Сгенерируй решение задачи: {input}"
)
gen_chain = LLMChain(llm=llm, prompt=gen_prompt, memory=memory_summary)

# Критик цепочка
critique_prompt = PromptTemplate(
    input_variables=["solution"],
    template="Критически оцени это решение и предложи улучшения:\n{solution}"
)
crit_chain = LLMChain(llm=llm, prompt=critique_prompt, memory=memory_summary)

def reflective_agent(user_input: str) -> str:
    # 1) Первичное решение
    solution = gen_chain.run(user_input)
    # 2) Критика и доработка
    critique = crit_chain.run({"solution": solution})
    if "недостаточно" in critique.lower():
        # если есть замечания — генерируем улучшенный вариант
        return gen_chain.run(user_input + "\nИсправь на основе: " + critique)
    return solution

# Пример:
print(reflective_agent("Напиши краткий план обучения по NLP для начинающих"))


/usr/local/lib/python3.11/dist-packages/langchain_community/llms/openai.py:469: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/usr/local/lib/python3.11/dist-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/usr/local/lib/python3.11/dist-packages/langchain_community/llms/openai.py:469: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/usr/local/lib/pyt

е. 

План обучения по NLP для начинающих:

**Уровень 1: Введение в NLP**

1.1. Определение NLP и его основные принципы.
1.2. Введение в моделирование, коммуникацию и изменение.
1.3. Практические упражнения: саморефлексия и осознание своих мыслей, эмоций и поведения.

**Уровень 2: Основы коммуникации**

2.1. Язык и коммуникация: вербальный и невербальный.
2.2. Активное слушание и сочувствие.
2.3. Практические упражнения: улучшение коммуникационных навыков в личных и профессиональных отношениях.

**Уровень 3: Понимание себя и других**

3.1. Понимание себя: ценности, убеждения и представления.
3.2. Понимание других: эмпатия и перспектива.
3.3. Практические упражнения: самосовершенствование и улучшение отношений.




/usr/local/lib/python3.11/dist-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


Here is the corrected code to initialize `SerpAPIWrapper` using the API key stored in Colab's Secrets Manager.